In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('Churn.csv')

C:\Users\leona\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,user,total_sessions,total_mediaids,total_days,total_played,android_app_time,ios_app_time,tv_app_time,mobile_web_time,desktop_web_time,...,total_active_dependents,sexo,idade,total_dependents,tipo_de_cobranca,assinatura_age,xEstado,xCidade,NumWeeks,churn
0,---3O9IoMUoaGDZG87g6OEDuuo-WYXaxN1pMOCyeqy4=,3,5,3,63.74,0.0,0.00,0.0,0.00,63.74,...,0,0,25.0,0,1,130.0,0,1,3,0
1,--0JRL6tA0PVx-sTQf35MjH5vZfK1fSi640oHXviUfU=,7,23,6,164.51,0.0,0.00,0.0,0.00,164.51,...,0,1,34.0,0,2,4176.0,4,2795,5,0
2,--0YC2EzVcnW7kkoTEwCvRips-nLwRzaL2OpOHAR1K8=,3,3,3,6.10,0.0,6.10,0.0,0.00,0.00,...,0,0,45.0,0,4,61.0,1,3989,3,0
3,--137VN7H4wWewQngTMnsQ==,6,12,6,167.76,0.0,167.52,0.0,0.24,0.00,...,0,1,64.0,5,2,5198.0,19,3915,5,0
4,--2wHGzXjzcr3fgcH4hIrwsI-ireVTjV3YA01_Qqf1E=,2,4,2,9.95,0.0,0.00,0.0,9.95,0.00,...,0,2,59.0,0,4,211.0,23,1002,2,0


In [4]:
# Estas features não serão consideradas para a previsão do Churn
df_Features= df.drop(['user','sexo','idade','total_dependents','tipo_de_cobranca','xEstado','xCidade'], axis=1)

In [5]:
#df_Features.to_csv('Globo_Com.csv',decimal='.',float_format='%.2f',encoding='utf-8', index=False)

In [6]:
len(df_Features)

561158

##### Análise do Balanceamento do Dataset

1- Data Sampling - Consiste na geração de novos exemplos positivos, ou na remoção de exemplos negativos, dada alguma heurística para balancear o conjunto de dados.

2- Cost Sensitive Learning - Atribui um custo para que o algoritmo aprenda que é muito pior perder um exemplo positivo do que um exemplo negativo.

3- Algorithm Adjustment - Modifica o Algoritmo de Aprendizado de Máquina para ser menos sensível a conjuntos de dados desbalanceados e sobrepostos a classes.


In [7]:
n_churn_0 = df_Features[df_Features['churn']==0]
n_churn_1 = df_Features[df_Features['churn']==1]
len(n_churn_0)


541643

In [8]:
len(n_churn_1)

19515

In [9]:
Percentual_Churn = (19515/541643)*100
Percentual_Churn

3.6029266509490565

##### Data Sampling

In [10]:
from sklearn.utils import resample

df_majority = df_Features[df_Features['churn']==0]
df_minority = df_Features[df_Features['churn']==1]

df_minority_upsampled = resample(df_minority,replace=True,
                        n_samples=541643, #Número de amostra da classe majoritária
                        random_state=1) # Seed value for resampling


In [11]:
df_Features_upsampled = pd.concat([df_majority, df_minority_upsampled])
df_Features_upsampled['churn'].value_counts()

1    541643
0    541643
Name: churn, dtype: int64

In [12]:
len(df_Features_upsampled)

1083286

##### Processando o Dataset Original

In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_Features, test_size = 0.20)

y_train = train['churn']
y_test = test['churn']

x_train = train
x_train = x_train.drop(columns = 'churn' )

x_test  = test
x_test  = x_test.drop(columns = 'churn' )

In [14]:
# Para instalar a Lib Graphviz : conda install python-graphviz
from sklearn import tree
import graphviz 
 
# Criando a Árvore de Decisão 
decisionTree = tree.DecisionTreeClassifier(max_depth = 4)
 
# Fit da Árvore 
decisionTree = decisionTree.fit(X=x_train, y=y_train)
 
# Gera a Árvore em PDF
churnTreeA = tree.export_graphviz(decisionTree, out_file=None, 
                         feature_names = list(x_train.columns.values),  
                         class_names = ['No Churn', 'Churn'],                                             
                         filled=True, rounded=True,  
                         special_characters=True) 





In [41]:
graphviz.__version__

'0.8.4'

##### Desenho (PDF) da Árvore

In [15]:
graph = graphviz.Source(churnTreeA)
graph.render('Globo_Churn_Tree.gv', view=True)

'Globo_Churn_Tree.gv.pdf'

### Métricas

In [36]:
import sklearn.metrics 
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
sklearn.__version__

'0.20.1'

In [37]:
y_pred = decisionTree.predict(x_test)

#### Acuracia

In [38]:
print('Accuracia no  test set: {:.2f}'.format(decisionTree.score(x_test, y_test)))

Accuracia no  test set: 0.97


#### F1 - Weighted

In [42]:
f1_score(y_test, y_pred, average='weighted')

0.9548818794057222

#### Balanced Accuracy

In [40]:
balanced_accuracy_score(y_test, y_pred)

0.5510471416120877